In [1]:
import sys
import os
import time
from collections import defaultdict
from PyQt5 import QtCore, QtWidgets
from PyQt5.QtWidgets import QFileDialog, QTreeWidgetItem, QVBoxLayout, QPushButton, QProgressBar, QApplication
from PyQt5.QtCore import QThread, QTimer, pyqtSignal

class Ui_MainWindow(object):
    def setupUi(self, MainWindow):
        MainWindow.setObjectName("MainWindow")
        MainWindow.resize(800, 600)
        self.centralwidget = QtWidgets.QWidget(MainWindow)
        self.centralwidget.setObjectName("centralwidget")
        self.verticalLayout = QtWidgets.QVBoxLayout(self.centralwidget)
        self.verticalLayout.setObjectName("verticalLayout")
        self.inputLayout = QtWidgets.QHBoxLayout()
        self.inputLayout.setObjectName("inputLayout")
        self.directoryLabel = QtWidgets.QLabel(self.centralwidget)
        self.directoryLabel.setObjectName("directoryLabel")
        self.inputLayout.addWidget(self.directoryLabel)
        self.directoryLineEdit = QtWidgets.QLineEdit(self.centralwidget)
        self.directoryLineEdit.setObjectName("directoryLineEdit")
        self.inputLayout.addWidget(self.directoryLineEdit)
        self.browseButton = QtWidgets.QPushButton(self.centralwidget)
        self.browseButton.setObjectName("browseButton")
        self.inputLayout.addWidget(self.browseButton)
        self.scanButton = QtWidgets.QPushButton(self.centralwidget)
        self.scanButton.setEnabled(False)
        self.scanButton.setObjectName("scanButton")
        self.inputLayout.addWidget(self.scanButton)
        self.stopButton = QtWidgets.QPushButton(self.centralwidget)
        self.stopButton.setVisible(False)
        self.stopButton.setObjectName("stopButton")
        self.inputLayout.addWidget(self.stopButton)
        self.verticalLayout.addLayout(self.inputLayout)
        self.treeWidget = QtWidgets.QTreeWidget(self.centralwidget)
        self.treeWidget.setObjectName("treeWidget")
        self.treeWidget.headerItem().setText(0, "File's name")
        self.treeWidget.headerItem().setText(1, "Size")
        self.treeWidget.headerItem().setText(2, "Folder")
        self.treeWidget.headerItem().setText(3, "Path")
        self.verticalLayout.addWidget(self.treeWidget)
        self.statusLayout = QtWidgets.QVBoxLayout()
        self.statusLayout.setObjectName("statusLayout")
        self.statusbar1 = QtWidgets.QLabel(self.centralwidget)
        self.statusbar1.setObjectName("statusbar1")
        self.statusLayout.addWidget(self.statusbar1)
        self.statusbar2 = QtWidgets.QLabel(self.centralwidget)
        self.statusbar2.setObjectName("statusbar2")
        self.statusLayout.addWidget(self.statusbar2)
        self.progressBar = QtWidgets.QProgressBar(self.centralwidget)
        self.progressBar.setVisible(False)
        self.statusLayout.addWidget(self.progressBar)
        self.verticalLayout.addLayout(self.statusLayout)
        MainWindow.setCentralWidget(self.centralwidget)

        self.retranslateUi(MainWindow)
        QtCore.QMetaObject.connectSlotsByName(MainWindow)

    def retranslateUi(self, MainWindow):
        _translate = QtCore.QCoreApplication.translate
        MainWindow.setWindowTitle(_translate("MainWindow", "File Dublicate Scanner"))
        self.directoryLabel.setText(_translate("MainWindow", "Directory:"))
        self.browseButton.setText(_translate("MainWindow", "Review"))
        self.scanButton.setText(_translate("MainWindow", "Scan"))
        self.stopButton.setText(_translate("MainWindow", "Stop"))

class ScanThread(QThread):
    update_progress = QtCore.pyqtSignal(int)
    scan_complete = QtCore.pyqtSignal(defaultdict)

    def __init__(self, directory, parent=None):
        self.directory = directory
        self.stop_requested = False
        super(ScanThread, self).__init__(parent)

    def run(self):
        file_sizes = defaultdict(list)
        total_files = sum([len(files) for r, d, files in os.walk(self.directory)])
        current_file = 0
        for root, dirs, files in os.walk(self.directory):
            if self.stop_requested:
                break
            for file in files:
                file_path = os.path.join(root, file)
                file_size = os.path.getsize(file_path)
                file_sizes[file_size].append((file_path, os.path.basename(root), file))
                current_file += 1
                progress = int((current_file / total_files) * 100)
                self.update_progress.emit(progress)
            if self.stop_requested:
                break

        if not self.stop_requested:
            self.scan_complete.emit(file_sizes)

    def stop(self):
        self.stop_requested = True

class MainWindow(QtWidgets.QMainWindow, Ui_MainWindow):
    def __init__(self, *args, **kwargs):
        QtWidgets.QMainWindow.__init__(self, *args, **kwargs)
        self.setupUi(self)

        self.progress_value = 0
        self.scan_thread = None
        self.timer = QTimer()
        self.timer.timeout.connect(self.update_progress_ui)
        self.timer.start(100)

        self.browseButton.clicked.connect(self.browse)
        self.scanButton.clicked.connect(self.scan)
        self.stopButton.clicked.connect(self.stop_scan)
        self.directoryLineEdit.textChanged.connect(self.enable_scan_button)


    def format_file_size(self, size):
        if size < 1024 * 1024:
            return f"{size / 1024:.2f} KB"
        else:
            return f"{size / 1024 / 1024:.2f} MB"

    def browse(self):
        directory = QFileDialog.getExistingDirectory()
        if directory:
            self.directoryLineEdit.setText(directory)

    def enable_scan_button(self):
        if self.directoryLineEdit.text():
            self.scanButton.setEnabled(True)
        else:
            self.scanButton.setEnabled(False)

    def paste_from_clipboard(self):
        clipboard = QtWidgets.QApplication.clipboard().text()
        if os.path.isdir(clipboard):
            self.directoryLineEdit.setText(clipboard)

    def scan(self):
        directory = self.directoryLineEdit.text()
        if directory:
            self.treeWidget.clear()
            self.statusbar1.clear()
            self.statusbar2.clear()
            self.progressBar.setMaximum(100)
            self.progressBar.setValue(0)
            self.progressBar.setVisible(True)
            self.scanButton.setVisible(False)
            self.stopButton.setVisible(True)
            self.scan_thread = ScanThread(directory)
            self.scan_thread.update_progress.connect(self.store_progress_value)
            self.scan_thread.scan_complete.connect(self.on_scan_complete)
            self.scan_thread.start()

    def stop_scan(self):
        if self.scan_thread:
            self.scan_thread.stop()
            self.stopButton.setVisible(False)
            self.scanButton.setVisible(True)

    def on_update_progress(self, progress, file_path, file_name):
        self.progress_value = int(progress)
        self.statusbar1.setText(f"Scan: {file_name}")
        self.statusbar2.setText(f"Path: {file_path}")
        QtCore.QCoreApplication.processEvents()

    def store_progress_value(self, progress):
        self.progress_value = progress

    def update_progress_ui(self):
        self.progressBar.setValue(self.progress_value)

    def on_scan_complete(self, file_sizes):
        self.progressBar.setValue(100)
        self.progressBar.setVisible(False)
        self.statusbar1.clear()
        self.statusbar2.clear()

        duplicates = 0
        duplicates_size = 0
        for size, files in file_sizes.items():
            if len(files) > 1:
                duplicates += len(files) - 1
                duplicates_size += (len(files) - 1) * size

        total_files = sum(len(files) for files in file_sizes.values())
        duplicates_percentage = (duplicates / total_files) * 100

        self.statusbar1.setText(f"Total Files: {total_files}, Dublicate Amount: {duplicates}")
        self.statusbar2.setText(
            f"Dublicate Size: {self.format_file_size(duplicates_size)}, Dublicate Percentage: {duplicates_percentage:.2f}%")

        for size, files in file_sizes.items():
            if len(files) > 1:
                original = QTreeWidgetItem(self.treeWidget)
                original.setText(0, files[0][2])
                original.setText(1, self.format_file_size(size))
                original.setText(2, files[0][1])
                original.setText(3, files[0][0])
                for duplicate in files[1:]:
                    dup = QTreeWidgetItem(original)
                    dup.setText(0, duplicate[2])
                    dup.setText(1, self.format_file_size(size))
                    dup.setText(2, duplicate[1])
                    dup.setText(3, duplicate[0])
                self.treeWidget.expandItem(original)

        self.treeWidget.collapseAll()

        self.stopButton.setVisible(False)
        self.scanButton.setVisible(True)



    def signal_accept(self, msg):

        self.pbar.setValue(int(msg))
        if self.pbar.value() == 100:
            self.pbar.setValue(0)
            self.btn.setEnabled(True)


class Thread(QThread):
    _signal = pyqtSignal(int)

    def __init__(self):
        super(Thread, self).__init__()

    def run(self):
        for i in range(101):
            time.sleep(0.1)
            self._signal.emit(i)

if __name__ == "__main__":
    app = QtWidgets.QApplication(sys.argv)
    MainWindow = MainWindow()
    MainWindow.show()
    sys.exit(app.exec_())

SystemExit: 0

C:\Users\NEW SKY\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3468: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
